In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from tkinter import font
from PIL import Image, ImageTk
import threading
import sys
import PIL
import math

# -- Library Version Check --
try:
    print("--- Checking Required Libraries ---")
    print(f"Python:     {sys.version.split()[0]}")
    print(f"OpenCV:     {cv2.__version__}")
    print(f"MediaPipe:  {mp.__version__}")
    print(f"NumPy:      {np.__version__}")
    print(f"Tkinter:    {tk.TkVersion}")
    print(f"Pillow:     {PIL.__version__}")
    print("---------------------------------")
except Exception as e:
    print(f"Error during version check: {e}")
    sys.exit(1)

class HandTrackerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Eucliance - Hand Gesture Recognition")
        self.geometry("1350x750")
        
        # --- Professional Color Palette & Fonts ---
        self.BG_COLOR = "#2c2c2c"        # Main background
        self.HEADER_COLOR = "#1a1a1a"    # Header background
        self.FRAME_COLOR = "#3e3e3e"     # Content frame background
        self.TEXT_BG_COLOR = "#1e1e1e"   # Data text background
        self.FG_COLOR = "#f0f0f0"        # Main text/foreground
        self.BORDER_COLOR = "#555555"    # Subtle border color

        # --- Define professional fonts ---
        
        self.font_header = ("Segoe UI", 14, "bold")
        self.font_title = ("Segoe UI", 16, "bold")
        
        # Consolas is a modern, clear monospace font for code/data.
        # If not found, it will fall back to Courier.
        self.font_data = font.Font(family="Consolas", size=10)
        
        self.configure(bg=self.BG_COLOR)
        # --- End of Style Definitions ---

        self.create_header()

        # MediaPipe Hands Init
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            max_num_hands=1,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )
        self.mp_drawing = mp.solutions.drawing_utils

        self.drawing_spec_dots = self.mp_drawing.DrawingSpec(
            color=(0, 255, 128), thickness=2, circle_radius=3)
        self.drawing_spec_lines = self.mp_drawing.DrawingSpec(
            color=(240, 240, 240), thickness=2, circle_radius=2)

        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Cannot open webcam.")
            self.destroy()
            return

        self.stop_event = threading.Event()
        self.create_widgets()

        self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
        self.video_thread.start()

        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def create_header(self):
        # Use the defined colors and fonts
        header_frame = tk.Frame(self, bg=self.HEADER_COLOR)
        # Added a bit more vertical padding
        header_frame.pack(fill=tk.X, padx=10, pady=(10, 5)) 
        
        tk.Label(header_frame, text="Project: Eucliance", font=self.font_header,
                 bg=self.HEADER_COLOR, fg=self.FG_COLOR, padx=15).pack(side=tk.LEFT)
        tk.Label(header_frame, text="By: Girija G", font=self.font_header,
                 bg=self.HEADER_COLOR, fg=self.FG_COLOR, padx=15).pack(side=tk.LEFT)
        tk.Label(header_frame, text="VIT Chennai", font=self.font_header,
                 bg=self.HEADER_COLOR, fg=self.FG_COLOR, padx=15).pack(side=tk.LEFT)

    def create_widgets(self):
        main_frame = tk.Frame(self, bg=self.BG_COLOR)
        # Added padding to separate from header and window edge
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=(5, 10))

        main_frame.grid_columnconfigure(0, weight=2) # Video frame gets more space
        main_frame.grid_columnconfigure(1, weight=1) # Data frame gets less
        main_frame.grid_rowconfigure(0, weight=1)

        # --- Video Frame ---
        # Changed relief from SUNKEN to SOLID for a flat, modern border
        video_frame = tk.Frame(main_frame, bg=self.FRAME_COLOR, bd=1, relief=tk.SOLID,
                               highlightbackground=self.BORDER_COLOR, highlightthickness=1)
        # Added consistent padding
        video_frame.grid(row=0, column=0, sticky="nsew", padx=(0, 10), pady=0)
        
        self.video_label = tk.Label(video_frame, bg=self.FRAME_COLOR)
        # Add padding *inside* the label to create a small margin
        self.video_label.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # --- Data Frame ---
        # Changed relief from SUNKEN to SOLID
        data_frame = tk.Frame(main_frame, bg=self.FRAME_COLOR, bd=1, relief=tk.SOLID,
                              highlightbackground=self.BORDER_COLOR, highlightthickness=1)
        data_frame.grid(row=0, column=1, sticky="nsew", padx=(10, 0), pady=0)
        
        # Use the defined title font and colors
        tk.Label(data_frame, text="Hand Landmarks & Distance", font=self.font_title,
                 bg=self.FRAME_COLOR, fg=self.FG_COLOR).pack(pady=15) # More top padding

        # Use the defined data font
        self.data_text = tk.Text(data_frame, wrap=tk.NONE, font=self.font_data,
                                 bg=self.TEXT_BG_COLOR, fg=self.FG_COLOR, bd=0, 
                                 highlightthickness=0, width=50)
        self.data_text.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        self.data_text.insert(tk.END, self.get_landmark_table_string(None, None))
        self.data_text.config(state=tk.DISABLED)

    def euclidean_distance(self, point1, point2):
        return math.sqrt(
            (point1.x - point2.x) ** 2 +
            (point1.y - point2.y) ** 2 +
            (point1.z - point2.z) ** 2)

    def video_loop(self):
        try:
            while not self.stop_event.is_set():
                success, frame = self.cap.read()
                if not success:
                    print("Ignoring empty frame.")
                    continue
                frame = cv2.flip(frame, 1)
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                rgb_frame.flags.writeable = False

                results = self.hands.process(rgb_frame)

                rgb_frame.flags.writeable = True

                hand_landmarks = None
                distance_thumb_index = None
                if results.multi_hand_landmarks:
                    hand_landmarks = results.multi_hand_landmarks[0]
                    self.mp_drawing.draw_landmarks(
                        rgb_frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                        self.drawing_spec_dots, self.drawing_spec_lines)

                    # Compute Euclidean distance between thumb tip (4) and index tip (8)
                    distance_thumb_index = self.euclidean_distance(
                        hand_landmarks.landmark[4], hand_landmarks.landmark[8])

                    # Draw bounding box
                    x_coords = [lm.x for lm in hand_landmarks.landmark]
                    y_coords = [lm.y for lm in hand_landmarks.landmark]
                    h, w, _ = rgb_frame.shape
                    xmin, xmax = int(min(x_coords) * w), int(max(x_coords) * w)
                    ymin, ymax = int(min(y_coords) * h), int(max(y_coords) * h)
                    # Use a slightly brighter color for the bounding box
                    cv2.rectangle(rgb_frame, (xmin, ymin), (xmax, ymax), (255, 50, 50), 2)

                    # Draw line between thumb tip and index tip
                    thumb_pos = hand_landmarks.landmark[4]
                    index_pos = hand_landmarks.landmark[8]
                    thumb_px = (int(thumb_pos.x * w), int(thumb_pos.y * h))
                    index_px = (int(index_pos.x * w), int(index_pos.y * h))
                    cv2.line(rgb_frame, thumb_px, index_px, (50, 255, 50), 3) # Bright Green

                    # Put distance text on the frame
                    mid_x = int((thumb_px[0] + index_px[0]) / 2)
                    mid_y = int((thumb_px[1] + index_px[1]) / 2)
                    distance_text = f"Dist: {distance_thumb_index:.4f}"
                    
                    # Add a small black background for better text readability
                    (text_w, text_h), _ = cv2.getTextSize(distance_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                    cv2.rectangle(rgb_frame, (mid_x + 5, mid_y - (text_h + 10)), (mid_x + 15 + text_w, mid_y), (0,0,0), -1)
                    
                    cv2.putText(rgb_frame, distance_text, (mid_x + 10, mid_y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50, 255, 50), 2)

                table_data = self.get_landmark_table_string(hand_landmarks, distance_thumb_index)

                img = Image.fromarray(rgb_frame)
                img_tk = ImageTk.PhotoImage(image=img)

                # Use .after() to safely update GUI from this thread
                self.after(0, self.update_gui, img_tk, table_data)

        except Exception as e:
            print(f"Error in video loop: {e}")
        finally:
            self.cap.release()

    def get_landmark_table_string(self, hand_landmarks, distance):
        header = (
            "=============================================================\n"
            " LANDMARK |     X     |     Y     |     Z     \n"
            "=============================================================\n"
        )
        if hand_landmarks is None:
            body = ""
            for i in range(21):
                body += f" {i:<9}| {'N/A':^11} | {'N/A':^11} | {'N/A':^11} \n"
            footer = "=============================================================\n" \
                     "                     NO HAND DETECTED                      \n" \
                     "============================================================="
            return header + body + footer

        body = ""
        for i, lm in enumerate(hand_landmarks.landmark):
            body += f" {i:<9}| {lm.x:^11.6f} | {lm.y:^11.6f} | {lm.z:^11.6f} \n"

        footer = "=============================================================\n"
        if distance is not None:
            footer += f" Distance (Thumb-Index): {distance:.6f} (normalized)"
        else:
            footer += " Distance (Thumb-Index): N/A"
        footer += "\n============================================================="
        return header + body + footer

    def update_gui(self, img_tk, table_data):
        if not self.stop_event.is_set():
            # Update video feed
            self.video_label.config(image=img_tk)
            self.video_label.image = img_tk
            
            # Update data table
            self.data_text.config(state=tk.NORMAL)
            self.data_text.delete(1.0, tk.END)
            self.data_text.insert(tk.END, table_data)
            self.data_text.config(state=tk.DISABLED)

    def on_closing(self):
        print("Closing application...")
        self.stop_event.set()
        if self.video_thread.is_alive():
            self.video_thread.join(timeout=1.0)
        self.destroy()

if __name__ == "__main__":
    app = HandTrackerApp()
    app.mainloop()